# ch15 估值框架
## 15.1 资产定价基本定理
资产定价基本定理是现代金融理论和数学的基石和成功案例之一。它的中心思想是鞅（Martingale）测度--也就是从折现后风险因素（随机过程）中消去漂移的概率测度。换言之，在鞅测度下，所有风险因素随无风险短期利率飘逸--而不包含某种无风险短期利率之上风险溢价的任何其他市场利率漂移。
### 15.1.1 简单示例
### 15.1.2 一般结果
## 15.2 风险中立折现
风险中立折现是风险中立估值方法的核心。因此，首先开发一个用于风险中立折现的**Python**类，但是首先仔细地观察一下估值**相关日期**的建模和处理时值得的。
### 15.2.1 日期建模和处理
折现的必要前提是建立日期模型。为了估值的目的，通常将当日和一般市场模型的最后日期T之间的间隔分为离散的时间间隔。这些时间间隔可以是同构的（等长），也可以是异构的（不等长）。估值库应该能够处理异构时间间隔的一般情况，因为更简单的情况会自动包含。因此，我们使用日期列表，假定最小的相关时间间隔为一天。这意味着，我们不关心日间事件--如果需要处理这些事件，就必须建立时间模型（除了日期之外）。

为了编辑一个相关日期列表，基本上可以采用两种方法：构建一个离散的日期列表（例如**Python**中的**datetime.datetime**对象）或者年份数（十进制数字，就像理论工作中常常做的）。

例如，下面的**dates**和**fraction**定义（大致）等价：

In [1]:
import datetime as dt
dates = [dt.datetime(2015,1,1),dt.datetime(2015,7,1),dt.datetime(2016,1,1)]

In [4]:
(dates[1]-dates[0]).days/365.

0.4958904109589041

In [5]:
(dates[2]-dates[1]).days/365.

0.5041095890410959

In [6]:
fractions=[0.0,0.5,1.0]

它们只是大致等价，因为年份数很少处于某一天的开始（0点）。

有时候，根据日期列表得出年份数是必要的，下例中介绍的**get_year_deltas**函数就是完成这项工作。

In [12]:
  #
  # 从datetime对象列表或者数组得出年份数的函数
  # get_year_deltas.py 
  #
import numpy as np
def get_year_deltas(date_list,day_count=365.):
    '''Return vector of floats with day deltas in years.
    Initial value normalized to zero.
    
    Parameters
    ===========
    date_list : list or array
        collection of datetime objects
    day_count : float
        number of days for a year
    (to account for different conventions)
Results
=======
delta_list : array
    yaer fractions
'''
    
    start = date_list[0]
    delta_list = [(date-start).days/day_count for date in date_list]
    return np.array(delta_list)

这个函数的应用如下：

In [10]:
import datetime as dt
dates = [dt.datetime(2015,1,1),dt.datetime(2015,7,1),dt.datetime(2016,1,1)]
get_year_deltas(dates)

array([ 0.        ,  0.49589041,  1.        ])

### 15.2.2 固定短期利率
我们将焦点放在短期利率折现最简单情况--也就是短期利率一直保持不变的情况。许多期权定价模型，如**Black-Scholes-Merton、Merton和Cox-Ross-Rubinstein**都做出这个假设。我们假定连续折现，因为这是期权定价应用中常见的假设。在这种情况下，给定未来日期t和固定短期利率r下，今天的一般折现因子可以由公式计算得出。

折现因子也可以解释为分别在t和T到期的单位零息债券（ZCB）的当日价值。

In [29]:
  # 
  # 固定短期利率下的风险中立折现类
  #constant_short_rate.py
  #
#from get_year_deltas import *
class constant_short_rate(object):
    '''Class for constant short rate discounting.
    
    Attributes
    ==========
    name : string
        name of the object
    short_rate : float(positive)
        constant rate for discounting
        
    Methods
    =======
get_discount_factors :
    get discount factors given a list/array of datetime objects or year fractions
'''
    def __init__(self,name,short_rate):
        self.name = name
        self.short_rate=short_rate
        if short_rate < 0:
            raise ValueError('short rate negative')
    
    def get_discount_factors(self,date_list,dtobjects=True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dlish = np.exp(self.short_rate * np.sort(-dlist))
        return np.array((date_list,dlist)).T

**constant_short_rate**类的应用：

In [30]:
import datetime as dt
dates = [dt.datetime(2015,1,1),dt.datetime(2015,7,1),dt.datetime(2016,1,1)]
#from constant_short_rate import *
csr = constant_short_rate('csr',0.05)
csr.get_discount_factors(dates)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.0],
       [datetime.datetime(2015, 7, 1, 0, 0), 0.4958904109589041],
       [datetime.datetime(2016, 1, 1, 0, 0), 1.0]], dtype=object)

主要的结果是一个二维**ndarry**对象，包含成对的**datetime**对象和相关折现因子。该类（特别是对象csr）通常也能处理年份数：

In [42]:
deltas = get_year_deltas(dates)
csr.get_discount_factors(deltas,dtobjects=False)

array([[ 0.        ,  0.        ],
       [ 0.49589041,  0.49589041],
       [ 1.        ,  1.        ]])

这个类负责其他类中需要的所有折现运算。

## 15.3 市场环境
市场环境“只是”其他数据和**Python**对象集合的一个名称。但是，使用这一抽象相当方便，因为它简化了许多操作，而且考虑到了循环特征的一致性建模。市场环境主要由3个字典组成，它们存储如下类型的数据和**Python**对象：

*常量*
    
    例如模型参数或者期权到期日。
*列表*
    
    一般是对象序列，例如用于建立（风险）证券模型对象的列表对象。
*曲线*

    用于折现的对象；例如，constant_short_rate类的实例。    

In [58]:
  #
  #建立由常量、列表和曲线组成的市场环境模型的类
  #market_environment.py
  #
class market_environment(object):
    '''Class to model a market environment relevant for valuation.
    
    Attributes
    ==========
    name:string
        name of the market environment
    pricing_date : datetime object
        date of the market environment
    
    Methods
    =======
    add_constant : 
        adds a constant(e.g. momdel parameter)
    get_constant :
        gets a constant
    add_list :
        adds a list(e.g. underlyings)
    get_list :
        gets a list
    add_curve : 
        adds a market curve(e.g. yield curve)
    get_curve :
        gets a market curve
    add_environment :
        adds and overwrites whole market environments
        with constants,lists,and curves
    '''
    
    def __init__(self,name,pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self,key,constant):
        self.constants[key] = constant
        
    def get_constant(self,key):
        return self.constants[key]
    
    def add_list(self,key,list_object):
        self.lists[key] = list_object
        
    def get_list(self,key):
        return self.lists[key]
    
    def add_curve(self,key,curve):
        self.curves[key] = curve
        
    def get_curve(self,key):
        return self.curves[key]
    
    def add_environment(self,env):
        # overwrites existing values,if they exist
        for key in env.constants:
            self.constants[key] = env.constants[key]
        for key in env.lists:
            self.lists[key] = env.lists[key]
        for key in env.curves:
            self.curves[key] = env.curves[key]

**market_environment**类的应用：

In [59]:
# from market_environment import *
import datetime as dt
dates = [dt.datetime(2015,1,1),dt.datetime(2015,7,1),dt.datetime(2016,1,1)]
csr = constant_short_rate('csr',0.05)
me_1 = market_environment('me_1',dt.datetime(2015,1,1))
me_1.add_list('symbols',['AAPL','MSFT','FB'])
me_1.get_list('symbols')

['AAPL', 'MSFT', 'FB']

In [60]:
me_2 = market_environment('me_2',dt.datetime(2015,1,1))
me_2.add_constant('volatility',0.2)
me_2.add_curve('short_rate',csr)
me_2.get_curve('short_rate')

In [61]:
me_1.add_environment(me_2)

In [62]:
me_1.get_curve('short_rate')

In [63]:
me_1.constants

{'volatility': 0.2}

In [64]:
me_1.lists

{'symbols': ['AAPL', 'MSFT', 'FB']}

In [65]:
me_1.curves

{'short_rate': <__main__.constant_short_rate at 0x87f96a0>}

In [66]:
me_1.get_curve('short_rate').short_rate

0.05

以上例子说明了对这个相当通用的“存储”类的基本处理。对于实用的应用程序，首先收集市场数据和其他数据以及**Python**对象，然后实例化**market_environment**对象并填入相关数据及对象。在这一步中，就可以将其交付给需要保存在对应**market_environment**中数据对象的其他类。

例如，这种面向对象建模方法的重大优势之一是，**constant_short_rate**类的实例可以存在于多个环境中。一旦实例被更新--例如，设置新的固定短期利率时--包含这个特殊折现类的所有**market_environment**实例都将被自动更新。
## 15.4 结语
本章为构建**Python**库，通过蒙特卡洛模拟估算期权和其他衍生品价值的较大项目提供一个框架。本章介绍资产定价基本定理，用相当简单的数值化示例加以阐述。在这一点上，为离散的一般市场模型提供了重要成果。

本章还为风险中立折现开发了一个**Python**类，以数值形式运用了资产定价基本定理的机制。根据**Python datetime**对象列表或者表示年份数的浮点对象，**constant_short_rate**类的实例提供对应的折现因子（单位零息债券的现值）。

本章的最后介绍了相当通用的**market_environment**类，该类集合了建模、模拟、估值和其他目的的相关数据和**Python**对象。

为了简化未来的导入工作，我们将使用一个包装器模块**dx_frame.py**：

下面的一条**import**语句可以在一步中使所有框架组件可用：
    
    from dx_frame import *

对于**Python**库和模块套件，还有一个选择是将所有相关**Python**模块保存在一个（子）目录，并在该目录中放入一个特殊的init文件进行所有导入工作。例如，将所有模块保存在dx子目录下：

在这种情况下，只要使用目录名就可以一次完成所有导入：

    from dx import *
也可以通过另一种方法：

    import dx
## 15.5 延伸阅读